In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn.functional as F
import torch.optim as optim

import os, sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../")))
from libs import data as dt, neuronshap as ns, sim
from cfgs.fedargs import *

from fairlearn.metrics import (
    demographic_parity_difference,
    demographic_parity_ratio,
    equalized_odds_difference,
    equalized_odds_ratio,
    false_negative_rate,
    false_positive_rate,
    true_negative_rate,
    true_positive_rate,
)
from libs.helpers.finance import bin_hours_per_week
from libs.helpers.metrics import (
    conditional_demographic_parity_difference,
    conditional_demographic_parity_ratio,
)
from libs.helpers.plot import group_box_plots

/Users/hkasyap/anaconda3/envs/menv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
names = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
    "salary",
]

In [3]:
def clean_string(s):
    """
    Helper function that strips leading / trailing whitespace, lower
    cases, and replaces hyphens with underscores.
    """
    return s.strip().lower().replace("-", "_")


def parse_native_country(country):
    """
    Group countries other than United-States and Mexico into single
    "other" category"
    """
    country = clean_string(country)
    if country == "united_states" or country == "mexico":
        return country
    return "other"

In [4]:
train = (
    pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
        header=None,
        na_values=[" ?"],
        names=names,
    )
    .drop(columns=["fnlwgt", "education_num"])
    # drop all rows with missing values
    .dropna()
    .reset_index(drop=True)
    # simple preprocessing on columns
    .assign(
        # clean all string columns
        education=lambda df: df.education.map(clean_string),
        marital_status=lambda df: df.marital_status.map(clean_string),
        occupation=lambda df: df.occupation.map(clean_string),
        race=lambda df: df.race.map(clean_string),
        relationship=lambda df: df.relationship.map(clean_string),
        workclass=lambda df: df.workclass.map(clean_string),
        # clean and aggregate native_country
        native_country=lambda df: df.native_country.map(parse_native_country),
        # encode binary features as integers
        salary=lambda df: (df.salary == " >50K").astype(np.int32),
        sex=lambda df: (df.sex == " Male").astype(np.int32),
    )
)

In [5]:
test = (
    pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
        header=None,
        na_values=[" ?"],
        skiprows=1,
        names=names,
    )
    .drop(columns=["fnlwgt", "education_num"])
    # drop all rows with missing values
    .dropna()
    .reset_index(drop=True)
    # simple preprocessing on columns
    .assign(
        # clean all string columns
        education=lambda df: df.education.map(clean_string),
        marital_status=lambda df: df.marital_status.map(clean_string),
        occupation=lambda df: df.occupation.map(clean_string),
        race=lambda df: df.race.map(clean_string),
        relationship=lambda df: df.relationship.map(clean_string),
        workclass=lambda df: df.workclass.map(clean_string),
        # clean and aggregate native_country
        native_country=lambda df: df.native_country.map(parse_native_country),
        # encode binary features as integers
        # note extra '.' in test set not present in train set
        salary=lambda df: (df.salary == " >50K.").astype(np.int32),
        sex=lambda df: (df.sex == " Male").astype(np.int32),
    )
)

In [6]:
assert set(train.education) == set(test.education)
assert set(train.race) == set(test.race)
assert set(train.relationship) == set(test.relationship)
assert set(train.marital_status) == set(test.marital_status)

In [7]:
one_hot_features = [
    "workclass",
    "education",
    "occupation",
    "race",
    "relationship",
    "marital_status",
    "native_country",
]

cts_features = ["age", "capital_gain", "capital_loss", "hours_per_week"]

binary_features = ["sex", "salary"]

In [8]:
train["race"].value_counts()

white                 25933
black                  2817
asian_pac_islander      895
amer_indian_eskimo      286
other                   231
Name: race, dtype: int64

In [9]:
train_df = pd.concat(
    [train, pd.get_dummies(train.loc[:, one_hot_features], dtype=np.int32)],
    axis=1,
)

test_df = pd.concat(
    [test, pd.get_dummies(test.loc[:, one_hot_features], dtype=np.int32)],
    axis=1,
)

In [10]:
assert train_df.columns.tolist() == test_df.columns.tolist()

In [11]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [12]:
data_dir = "../../data/adult"

In [13]:
original_features = cts_features + one_hot_features + binary_features

train_df[original_features].to_csv("../../data/adult/train.csv", index=False)
val_df[original_features].to_csv("../../data/adult/val.csv", index=False)
test_df[original_features].to_csv("../../data/adult/test.csv", index=False)

In [14]:
ss = StandardScaler()

train_df[cts_features] = ss.fit_transform(train_df[cts_features])
val_df[cts_features] = ss.transform(val_df[cts_features])
test_df[cts_features] = ss.transform(test_df[cts_features])

In [15]:
train_df.drop(columns=one_hot_features).to_csv("../../data/adult/train-one-hot.csv", index=False)
val_df.drop(columns=one_hot_features).to_csv("../../data/adult/val-one-hot.csv", index=False)
test_df.drop(columns=one_hot_features).to_csv("../../data/adult/test-one-hot.csv", index=False)

In [16]:
train = pd.read_csv("../../data/adult/train.csv")
val = pd.read_csv("../../data/adult/val.csv")
test = pd.read_csv("../../data/adult/test.csv")

train_oh = pd.read_csv("../../data/adult/train-one-hot.csv")
val_oh = pd.read_csv("../../data/adult/val-one-hot.csv")
test_oh = pd.read_csv("../../data/adult/test-one-hot.csv")

In [17]:
test_oh.head(5)

,age,sex,capital_gain,capital_loss,hours_per_week,salary,workclass_federal_gov,workclass_local_gov,workclass_private,workclass_self_emp_inc,...,marital_status_divorced,marital_status_married_af_spouse,marital_status_married_civ_spouse,marital_status_married_spouse_absent,marital_status_never_married,marital_status_separated,marital_status_widowed,native_country_mexico,native_country_other,native_country_united_states
0,-1.015917,1,-0.147741,-0.218133,-0.079269,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
1,-0.029378,1,-0.147741,-0.218133,0.752765,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
2,-0.788255,1,-0.147741,-0.218133,-0.079269,1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
3,0.425948,1,0.872159,-0.218133,-0.079269,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
4,-0.332929,1,-0.147741,-0.218133,-0.911303,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1


In [18]:
#https://github.com/ritvikkhanna09/Census-classifier-comparison

In [19]:
'''
race_amer_indian_eskimo               15060 non-null  int64  
race_asian_pac_islander               15060 non-null  int64  
race_black                            15060 non-null  int64  
race_other                            15060 non-null  int64  
race_white
 
'''

mr_dh_oh = test_oh.loc[(test_oh["race_asian_pac_islander"] == 1) | (test_oh["race_white"] == 1)]
mr_dh_oh = mr_dh_oh.head(200)
fmr_dh_oh = test_oh.loc[(test_oh["race_amer_indian_eskimo"] == 1) | (test_oh["race_black"] == 1) | (test_oh["race_other"] == 1)]
fmr_dh_oh = fmr_dh_oh.head(200)


m_dh_oh = test_oh.loc[test_oh["sex"] == 1]
m_dh_oh = m_dh_oh.head(200)
fm_dh_oh = test_oh.loc[test_oh["sex"] == 0]
fm_dh_oh = fm_dh_oh.head(200)

In [20]:
X_train = train_oh.drop(columns="salary").values
Y_train = train_oh['salary'].values
X_test = test_oh.drop(columns="salary").values
Y_test = test_oh['salary'].values
X_m = m_dh_oh.drop(columns="salary").values
Y_m = m_dh_oh['salary'].values
X_fm = fm_dh_oh.drop(columns="salary").values
Y_fm = fm_dh_oh['salary'].values

X_mr = mr_dh_oh.drop(columns="salary").values
Y_mr = mr_dh_oh['salary'].values
X_fmr = fmr_dh_oh.drop(columns="salary").values
Y_fmr = fmr_dh_oh['salary'].values

#creating torch dataset and loader using original dataset. 
#to use resampled dataset, replace ex. xtrain with xtrain_over etc.
train_data = torch.utils.data.TensorDataset(torch.tensor(X_train).float(), torch.tensor(Y_train).long())
test_data = torch.utils.data.TensorDataset(torch.tensor(X_test).float(), torch.tensor(Y_test).long())
m_data = torch.utils.data.TensorDataset(torch.tensor(X_m).float(), torch.tensor(Y_m).long())
fm_data = torch.utils.data.TensorDataset(torch.tensor(X_fm).float(), torch.tensor(Y_fm).long())

mr_data = torch.utils.data.TensorDataset(torch.tensor(X_mr).float(), torch.tensor(Y_mr).long())
fmr_data = torch.utils.data.TensorDataset(torch.tensor(X_fmr).float(), torch.tensor(Y_fmr).long())

train_loader = torch.utils.data.DataLoader(train_data,batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=len(test_data))
m_loader = torch.utils.data.DataLoader(m_data, batch_size=1)
fm_loader = torch.utils.data.DataLoader(fm_data, batch_size=1)
mr_loader = torch.utils.data.DataLoader(mr_data, batch_size=1)
fmr_loader = torch.utils.data.DataLoader(fmr_data, batch_size=1)

In [21]:
class BasicNet(torch.nn.Module):
    
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.num_features = num_features
        self.num_classes = num_classes
        self.layers = 0
        
        self.lin1 = torch.nn.Linear(self.num_features,  150)        
        self.lin2 = torch.nn.Linear(50, 50)        
        self.lin3 = torch.nn.Linear(50, 50)
        
        self.lin4 = torch.nn.Linear(150, 150) 
        
        self.lin5 = torch.nn.Linear(50, 50)        
        self.lin6 = torch.nn.Linear(50, 50)
        self.lin10 = torch.nn.Linear(150, self.num_classes)
        
        self.prelu = torch.nn.PReLU()
        self.dropout = torch.nn.Dropout(0.25)

    def forward(self, xin):
        self.layers = 0
        
        x = F.relu(self.lin1(xin))
        self.layers += 1
        
        #x = F.relu(self.lin2(x))
        #self.layers += 1
        for y in range(8):
            x = F.relu(self.lin4(x)) 
            self.layers += 1
           
        x = self.dropout(x)
        
        x = F.relu(self.lin10(x)) 
        self.layers += 1
        return x

In [22]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    
    for inputs, target in train_loader:
      
        #inputs, target = inputs.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target.long())
        # Backprop
        loss.backward()
        optimizer.step()
        ###

In [23]:
def test(model, test_loader):
    model.eval()
    
    test_loss = 0
    correct = 0
    test_size = 0
    
    with torch.no_grad():
      
        for inputs, target in test_loader:
            
            #inputs, target = inputs.to(device), target.to(device)
            
            output = model(inputs)
            test_size += len(inputs)
            test_loss += test_loss_fn(output, target.long()).item() 
            pred = output.max(1, keepdim=True)[1] 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= test_size
    accuracy = correct / test_size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, test_size,
        100. * accuracy))
    
    return test_loss, accuracy

In [24]:
model = BasicNet(63, 2)
test_accuracy = []
train_loss = []
nbr_epochs = 5
lr = 0.0025# 
weight_decay = 0

# Surrogate loss used for training
loss_fn = torch.nn.CrossEntropyLoss()
test_loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')

optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)
#optimizer = optim.SGD(model.parameters(), lr=lr ,weight_decay=weight_decay)
#optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)

print('Training beginning...')
#start_time = time.time()

for epoch in range(1, nbr_epochs+1):
    print('Epoch ', epoch, ':')
    train(model, train_loader, optimizer, epoch)
    loss, acc = test(model, test_loader)
    
    # save results every epoch
    test_accuracy.append(acc)
    train_loss.append(loss)
    
#end_time = time.time()
#print('Training on ' + str(nbr_epochs) + ' epochs done in ', str(end_time-start_time),' seconds')

Training beginning...
Epoch  1 :

Test set: Average loss: 0.3584, Accuracy: 11360/15060 (75%)

Epoch  2 :

Test set: Average loss: 0.3356, Accuracy: 12710/15060 (84%)

Epoch  3 :

Test set: Average loss: 0.3398, Accuracy: 12798/15060 (85%)

Epoch  4 :

Test set: Average loss: 0.3413, Accuracy: 12812/15060 (85%)

Epoch  5 :

Test set: Average loss: 0.3212, Accuracy: 12795/15060 (85%)



In [25]:
with torch.no_grad():
    for inputs, target in test_loader:
        outputs = model(inputs)
        pred = outputs.max(1, keepdim=True)[1] 
        correct = pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / len(inputs)
        print('\nAccuracy: {}/{} ({:.0f}%)\n'.format(correct, len(inputs), 100. * accuracy))

Y_prob = F.softmax(outputs, dim=1)[:, 1]
Y_pred = outputs.max(1, keepdim=True)[1]

print(sum(Y_test), sum(Y_pred), sum(pred))

test = pd.read_csv("../../data/adult/test.csv")


Accuracy: 12795/15060 (85%)

3700 tensor([3505]) tensor([3505])


<h1>Demographic Parity</h1>

<h2>Distribution of scores by sex</h2>

In [131]:
dpd = demographic_parity_difference(
    Y_test, Y_pred, sensitive_features=test.sex,
)
dpr = demographic_parity_ratio(
    Y_test, Y_pred, sensitive_features=test.sex,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.016
Demographic parity ratio: 0.360


<h2>Distribution of scores by race</h2>

In [132]:
dpd = demographic_parity_difference(
    Y_test, Y_pred, sensitive_features=test.race,
)
dpr = demographic_parity_ratio(
    Y_test, Y_pred, sensitive_features=test.race,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.021
Demographic parity ratio: 0.000


<h1>Conditional Demographic Parity</h1>

<h2>Distribution of scores by sex and hours worked per week</h2>

In [133]:
test_hpw_enum = test.hours_per_week.map(bin_hours_per_week)

cdpd = conditional_demographic_parity_difference(
    Y_test, Y_pred, test.sex, test_hpw_enum,
)
cdpr = conditional_demographic_parity_ratio(
    Y_test, Y_pred, test.sex, test_hpw_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

Conditional demographic parity difference: 0.012
Conditional demographic parity ratio: 0.640


<h2>Distribution of scores by race and hours worked per week</h2>

In [134]:
cdpd = conditional_demographic_parity_difference(
    Y_test, Y_pred, test.race, test_hpw_enum,
)
cdpr = conditional_demographic_parity_ratio(
    Y_test, Y_pred, test.race, test_hpw_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

Conditional demographic parity difference: 0.040
Conditional demographic parity ratio: 0.000


<h1>Equalised Odds</h1>

<h2>Distribution of scores by sex for high and low earners</h2>

In [135]:
eod = equalized_odds_difference(
    Y_test, Y_pred, sensitive_features=test.sex,
)
eor = equalized_odds_ratio(
    Y_test, Y_pred, sensitive_features=test.sex,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

Equalised odds difference: 0.000
Equalised odds ratio: 0.000


<h2>Distribution of scores by race for high and low earners</h2>

In [136]:
eod = equalized_odds_difference(
    Y_test, Y_pred, sensitive_features=test.race,
)
eor = equalized_odds_ratio(
    Y_test, Y_pred, sensitive_features=test.race,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

Equalised odds difference: 0.083
Equalised odds ratio: 0.000


<h1>Shapley based Neuron Pruning for Fairness</h1>

In [76]:
m_shapley_values = ns.calculate_shapley_values_fa(model, m_loader, 200)
print(m_shapley_values)
fm_shapley_values = ns.calculate_shapley_values_fa(model, fm_loader, 200)
print(fm_shapley_values)

[1.8187975e+01 1.0167586e+01 2.2399819e+00 ... 3.0218979e+03 1.4633556e+03
 0.0000000e+00]
[1.8956179e+01 0.0000000e+00 2.4627926e+00 ... 2.8495352e+03 5.6987415e+02
 0.0000000e+00]


In [116]:
diff_shap_values = m_shapley_values - fm_shapley_values
max_diff_shap_values_ind = np.argpartition(diff_shap_values, -150)[-150:]
diff_shap_values[max_diff_shap_values_ind]

array([ 20.514679,  20.552254,  20.525959,  20.540623,  20.675745,
        21.4833  ,  23.102304,  29.133503,  31.949133,  33.825687,
        36.029438,  27.950514,  22.26446 ,  23.345766,  34.53684 ,
        20.965221,  25.527058,  22.258518,  21.061516,  28.421902,
        22.078074,  23.240614,  23.663237,  29.969898,  22.62664 ,
        31.19026 ,  22.220913,  24.988935,  26.228916,  23.653927,
        26.74673 ,  22.462141,  34.473793,  33.311325,  29.188107,
        22.994347,  25.133823,  33.23652 ,  24.184479,  25.95591 ,
        23.615437,  22.99022 ,  20.905025,  21.19911 ,  21.08009 ,
        31.478962,  21.82454 ,  30.615107,  21.727694,  31.345827,
        22.690218,  28.208124,  26.3876  ,  33.637077,  35.9862  ,
        26.786005,  24.202766,  30.439253,  26.432623,  31.194931,
        31.762396,  21.9769  ,  23.275398,  34.305424,  26.649963,
        22.0402  ,  27.956675,  31.016626,  27.516064,  32.82853 ,
        34.564808,  34.95449 ,  24.102848,  25.45012 ,  21.079

In [117]:
model_arr, model_slist = sim.get_net_arr(model)
model_arr[max_diff_shap_values_ind] = 0
updated_model = sim.get_arr_net(model, model_arr, model_slist)

In [118]:
mr_shapley_values = ns.calculate_shapley_values_fa(updated_model, mr_loader, 200)
print(mr_shapley_values)
fmr_shapley_values = ns.calculate_shapley_values_fa(updated_model, fmr_loader, 200)
print(fmr_shapley_values)

[14.007823   4.891384   1.5297899 ...  0.         0.         0.       ]
[27.33152    8.849223   3.5181758 ...  0.         0.         0.       ]


In [128]:
diff_shap_values = mr_shapley_values - fmr_shapley_values
max_diff_shap_values_ind = np.argpartition(diff_shap_values, -200)[-200:]
diff_shap_values[max_diff_shap_values_ind]

array([ 2.0578184,  2.070988 ,  2.0867162,  2.2250242,  2.185326 ,
        2.0956097,  2.1704085,  2.19269  ,  2.2700267,  2.199029 ,
        2.2031686,  2.1103187,  2.2087085,  2.1521459,  2.1287103,
        2.133597 ,  2.280058 ,  2.299098 ,  2.1941013,  2.1722193,
        2.326398 ,  3.6522806,  2.6364245,  2.5335124,  4.8151093,
        6.0289803,  2.4403512,  2.8562846, 11.405832 ,  7.800834 ,
        8.42189  ,  4.6132274, 17.267637 ,  2.6602168,  2.996046 ,
        2.7469487,  6.6800647,  6.103897 ,  8.912223 ,  6.111103 ,
        8.68351  ,  3.0433805, 23.208132 ,  6.2871833, 26.14472  ,
        7.120043 ,  3.6457086,  2.5271273,  4.627925 ,  3.2775664,
        2.7457848,  6.982445 ,  8.315563 ,  2.6586857,  4.5852065,
        3.7395742,  3.966404 ,  4.027649 ,  3.0975237, 61.81136  ,
       10.170212 , 14.007952 ,  2.3797092,  3.0894146,  2.755583 ,
       12.433205 ,  2.5990474,  6.734474 , 20.655739 ,  4.006357 ,
       10.616814 ,  4.910745 ,  2.559756 ,  5.5025826,  2.5762

In [129]:
model_arr, model_slist = sim.get_net_arr(updated_model)
model_arr[max_diff_shap_values_ind] = 0
updated_model_2 = sim.get_arr_net(model, model_arr, model_slist)

In [130]:
with torch.no_grad():
    for inputs, target in test_loader:
        outputs = updated_model_2(inputs)
        pred = outputs.max(1, keepdim=True)[1] 
        correct = pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / len(inputs)
        print('\nAccuracy: {}/{} ({:.0f}%)\n'.format(correct, len(inputs), 100. * accuracy))
        

Y_prob = F.softmax(outputs, dim=1)[:, 1]
Y_pred = outputs.max(1, keepdim=True)[1]

print(sum(Y_test), sum(Y_pred), sum(pred))

test = pd.read_csv("../../data/adult/test.csv")


Accuracy: 11644/15060 (77%)

3700 tensor([290]) tensor([290])
